In [1]:
import os
import sys
import math

sys.path.append('../')

In [2]:
import torch
import pytorch_lightning as pl
from torchinfo import summary
from pytorch_lightning.loggers import TensorBoardLogger

In [3]:
from benchmark.data import read_tep_data
from benchmark.deep_learning import TimeSeriesClassifier
from benchmark.deep_learning import FullyConvolutionalEncoder

## Reading the data

In [4]:
sources = [1, 3, 4, 5, 6]
target = [2,]

In [5]:
Xs, ys = read_tep_data(
    base_path='../tep_data/benchmark/',
    modes=sources,
    normalization="standardization",
    return_domain=False,
    as_list=False,
    channels_first=False,
    one_hot_labels=False
)

In [6]:
Xs.shape, ys.shape

(torch.Size([14444, 600, 34]), torch.Size([14444]))

In [7]:
Xt, yt = read_tep_data(
    base_path='../tep_data/benchmark/',
    modes=target,
    normalization="standardization",
    return_domain=False,
    as_list=False,
    channels_first=False,
    one_hot_labels=False
)

In [8]:
Xt.shape, yt.shape

(torch.Size([2845, 600, 34]), torch.Size([2845]))

In [9]:
src_dataset = torch.utils.data.TensorDataset(Xs, ys)
src_dataloader = torch.utils.data.DataLoader(
    src_dataset, batch_size=128, shuffle=True)

tgt_dataset = torch.utils.data.TensorDataset(Xt, yt)
tgt_dataloader = torch.utils.data.DataLoader(
    tgt_dataset, batch_size=128, shuffle=True)

## Creating the neural net

In [10]:
34 * 16

544

In [11]:
class PositionalEncoding(torch.nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [12]:
nhead = 8
head_dim = 32
d_model = nhead * head_dim

embedding_layer = torch.nn.Linear(34, d_model)
pos_encoding = PositionalEncoding(emb_size=d_model, dropout=0.1, maxlen=600)

layer1 = torch.nn.TransformerEncoderLayer(
    d_model=d_model,
    nhead=nhead,
    dim_feedforward=512,
    dropout=0.1,
    activation='relu',
    batch_first=True
)

In [13]:
embedding = embedding_layer(Xs[:16])
embedding = pos_encoding(embedding)

In [14]:
print(embedding.shape)

torch.Size([16, 600, 256])


In [15]:
class TransformerEncoder(torch.nn.Module):
    def __init__(self, d_input, head_dim, nhead, dim_feedforward, dropout=0.1, seq_len=600,
                 num_layers=6, reduction='avg'):
        super(TransformerEncoder, self).__init__()
        d_model = head_dim * nhead
        self.embedding_layer = torch.nn.Linear(d_input, d_model)
        self.pos_encoding = PositionalEncoding(emb_size=d_model, dropout=dropout, maxlen=seq_len)
        layer = torch.nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu',
            batch_first=True
        )
        self.transformer = torch.nn.TransformerEncoder(
            layer, num_layers
        )
        assert reduction.lower() in ['last', 'avg']
        self.reduction = reduction.lower()
        
    def forward(self, x):
        e = self.pos_encoding(
            self.embedding_layer(x))
        h = self.transformer(e)
        
        if self.reduction == 'last':
            h = h[:, -1, :]
        if self.reduction == 'avg':
            h = h.mean(dim=1)
        return h

In [16]:
encoder = TransformerEncoder(
    d_input=34,
    head_dim=16,
    nhead=8,
    dim_feedforward=256,
    dropout=0.1,
    seq_len=600,
    num_layers=6,
    reduction='last'
)

In [17]:
summary(encoder, input_shape=(16, 600, 34))

Layer (type:depth-idx)                                            Param #
TransformerEncoder                                                --
├─Linear: 1-1                                                     4,480
├─PositionalEncoding: 1-2                                         --
│    └─Dropout: 2-1                                               --
├─TransformerEncoder: 1-3                                         --
│    └─ModuleList: 2-2                                            --
│    │    └─TransformerEncoderLayer: 3-1                          132,480
│    │    └─TransformerEncoderLayer: 3-2                          132,480
│    │    └─TransformerEncoderLayer: 3-3                          132,480
│    │    └─TransformerEncoderLayer: 3-4                          132,480
│    │    └─TransformerEncoderLayer: 3-5                          132,480
│    │    └─TransformerEncoderLayer: 3-6                          132,480
Total params: 799,360
Trainable params: 799,360
Non-trainable par

In [18]:
encoder(Xs[:16]).shape

torch.Size([16, 128])

In [19]:
clf = torch.nn.Linear(128, 29)

## Training with Pytorch Lightning

In [20]:
n_epochs = 70
momentum = 0.9
l2_penalty = 0.0
learning_rate = 1e-4
optimizer_name = 'adam'

In [21]:
model = TimeSeriesClassifier(encoder=encoder,
                             clf=clf,
                             n_classes=29,
                             loss_fn=torch.nn.CrossEntropyLoss(),
                             input_shape=(Xs.shape[1:]),
                             learning_rate=learning_rate,
                             l2_penalty=l2_penalty,
                             momentum=momentum,
                             optimizer_name=optimizer_name)

In [22]:
log_dir = f"../models/logs/{sources}->{target[0]}"
log_name = "multi_source_only_baseline_transformer_last"

In [23]:
logger = TensorBoardLogger(save_dir=log_dir, name=log_name)

In [24]:
trainer = pl.Trainer(
    max_epochs=n_epochs,
    accelerator='gpu',
    logger=logger,
    enable_checkpointing=True,
    enable_progress_bar=True)
trainer.fit(model,
            src_dataloader,
            tgt_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: ../models/logs/[1, 3, 4, 5, 6]->2/multi_source_only_baseline_transformer_last
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type               | Params | In sizes      | Out sizes
---------------------------------------------------------------------------
0 | clf     | Linear             | 3.7 K  | [16, 128]     | [16, 29] 
1 | encoder | TransformerEncoder | 799 K  | [16, 600, 34] | [16, 128]
2 | loss_fn | CrossEntropyLoss   | 0      | ?     

Sanity Checking: 0it [00:00, ?it/s]

/home/efernand/anaconda3/envs/OptimalTransport/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/home/efernand/anaconda3/envs/OptimalTransport/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/efernand/anaconda3/envs/OptimalTransport/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/efernand/anaconda3/envs/OptimalTransport/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
